In [1]:
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.types.{StructType, IntegerType, StringType, StructField}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import io.hops.util.Hops
import io.github.ackuq.pit._

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
40,application_1642582607798_0039,spark,idle,Link,Link


SparkSession available as 'spark'.
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.types.{StructType, IntegerType, StringType, StructField}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import io.hops.util.Hops
import io.github.ackuq.pit._


In [2]:
val BASE_PATH = "hdfs:///Projects/" + Hops.getProjectName + "/Jupyter/PIT-joins/experiment/data/"
val SIZES
val DATA_DIRECTORIES = Seq("10000-1_year", "20000-1_year", "40000-1_year", "80000-1_year")

BASE_PATH: String = hdfs:///Projects/demo_fs_meb10000/Jupyter/PIT-joins/experiment/data/
DATA_DIRECTORIES: Seq[String] = List(10000-1_year, 20000-1_year, 40000-1_year, 80000-1_year)


In [3]:
val labelData = spark.read.parquet(BASE_PATH + DATA_DIRECTORIES(0) + "/left.parquet")
val featureData = spark.read.parquet(BASE_PATH + DATA_DIRECTORIES(0) + "/right.parquet")

labelData: org.apache.spark.sql.DataFrame = [id: bigint, ts: bigint ... 1 more field]
featureData: org.apache.spark.sql.DataFrame = [id: bigint, ts: bigint ... 1 more field]


In [4]:
def unionPIT(label: DataFrame, features: DataFrame): DataFrame = {
    Union.join(
        label,
        features,
        leftPrefix = Some("df1_"),
        rightPrefix = "df2_",
        partitionCols = Seq("id")
    )
}

unionPIT: (label: org.apache.spark.sql.DataFrame, features: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [5]:
// Test the union PIT
val pitTestUnion = unionPIT(labelData, featureData)

pitTestUnion: org.apache.spark.sql.DataFrame = [id: bigint, df1_ts: bigint ... 3 more fields]


In [6]:
pitTestUnion.show

+----+--------+---------+--------+---------+
|  id|  df1_ts|df1_label|  df2_ts|df2_value|
+----+--------+---------+--------+---------+
|  26|24685843|       26|24679690|       26|
|  29|10406545|       29|10101990|       29|
| 964| 8806710|      964| 8019480|      964|
|1677|18761519|     1677|18227808|     1677|
|1697|23753009|     1697|18146106|     1697|
|1806|10608320|     1806|10357110|     1806|
|2214| 6138051|     2214| 5260102|     2214|
|2250|13310809|     2250|11004066|     2250|
|2453|11537917|     2453|10005059|     2453|
|2509| 2333796|     2509| 2066738|     2509|
|2529| 6809193|     2529| 4466097|     2529|
|3091|25861160|     3091|25702301|     3091|
|3506|21356807|     3506|21248555|     3506|
|3764|27585395|     3764|20872490|     3764|
|4590|20714029|     4590|20711708|     4590|
|4823| 4141623|     4823| 2692286|     4823|
|4894| 2667262|     4894| 2281705|     4894|
|5385|22802515|     5385|22649902|     5385|
|5409|18079144|     5409|11639738|     5409|
|5556| 857

In [ ]:
pitTestUnion.count

In [ ]:
def explodingPIT(label: DataFrame, features: DataFrame): DataFrame = {
    Exploding.join(
        label,
        features,
        leftTSColumn = label("ts"),
        rightTSColumn = features("ts"),
        partitionCols = Seq((label("id"), features("id")))
    )
}

In [ ]:
val pitTestExploding = explodingPIT(labelData, featureData)

In [ ]:
pitTestExploding.show

In [ ]:
pitTestExploding.count

In [ ]:
EarlyStopSortMerge.init(spark)

def earlyStopPIT(label: DataFrame, features: DataFrame): DataFrame = {
    label.join(features, (
            EarlyStopSortMerge.pit(label("ts"), features("ts"))
        ) && (label("id") === features("id"))
    )
}

In [ ]:
val pitTestEarlyStop = earlyStopPIT(labelData, featureData)

In [ ]:
pitTestEarlyStop.show

In [ ]:
pitTestEarlyStop.count